_Dies ist ein Spickzettel für diverse jQAssistant / Cypher Codeschnipsel_

# jQAssistant Demos
* Neo4j-Server starten
 * `mvn jqassistant:server`
* Browser öffnen
 * http://localhost:7474/browser/
* Drawer öffnen
* Labels durchklicken
 * `Commit`
 * `Class`
 * `:DECLARES`
 * jQAssistant Dokumentation: http://buschmais.github.io/jqassistant/doc/1.3.0/#_java_plugin
 
## Beispiel-Queries


**Infos über Verbindungen eines Knoten**
```cypher
MATCH
  (c:Class {name:"Pet"})-[outgoing]->(out)
RETURN c.name as Klasse,  labels(c) as KlassenLabels, "-[:" + type(outgoing) + "]->" as Verbindungstyp, count(outgoing) as Anzahl, labels(out) as KnotenLabels
ORDER BY Anzahl DESC, Verbindungstyp
UNION
MATCH 
  (in)-[incoming]->(c:Class {name:"Pet"})
RETURN c.name as Klasse, labels(c) as KlassenLabels,  "<-[:" + type(incoming) + "]-" as Verbindungstyp, count(incoming) as Anzahl, labels(in) as KnotenLabels
ORDER BY Anzahl DESC, Verbindungstyp
```


**Klasse mit den meisten Methoden**
```cypher
MATCH 
  (c:Class)-[:DECLARES]->(m:Method)
RETURN c.fqn as Klasse, COUNT(m.signature) as Methoden
ORDER BY Methoden DESC
```
**Rekursive Aufrufe**
```cypher
MATCH
  (c:Class)-[:DECLARES]->(m:Method)-[:INVOKES*1..5]->(m)
RETURN c, m
```

**Rekursive Aufrufe zur Datenbank**
```cypher
MATCH 
  (m:Method)-[:INVOKES*]->(m)-[:INVOKES]->(dbMethod:Method),
  (dbMethod)<-[:DECLARES]-(dbClass:Class)
WHERE dbClass.name ENDS WITH "Database"
RETURN m, dbMethod, dbClass
```
**Statische, geschriebene Variablen**
```cypher
MATCH (c:Class)-[:DECLARES]->(f:Field)<-[w:WRITES]-(m:Method)
WHERE 
    EXISTS(f.static) AND NOT EXISTS(f.final)
RETURN 
    c.name as InClass, 
    m.name as theMethod, 
    w.lineNumber as writesInLine, 
    f.name as toStaticField
```
* [Analyse Notebook](./Race%20Condition%20Analysis%20Demo%20Notebook.ipynb)

**Aggregation von Messergebnissen über fachliche Bereiche**
```cypher
MATCH 
  (t:Type)-[:BELONGS_TO]->(s:Subdomain),
  (t)-[:HAS_CHANGE]->(ch:Change),
  (t)-[:HAS_MEASURE]->(co:Coverage)
OPTIONAL MATCH
  (t)-[:HAS_BUG]->(b:BugInstance)   
RETURN 
  s.name as ASubdomain,
  COUNT(DISTINCT t) as Types,
  COUNT(DISTINCT ch) as Changes,
  AVG(co.ratio) as Coverage,
  COUNT(DISTINCT b) as Bugs
ORDER BY Coverage ASC, Bugs DESC
```
* [Analyse Notebook](./Strategic%20Redesign.ipynb)